In [ ]:
import torchaudio
from nemo.collections.asr.models import EncDecCTCModel

from utils import stt
import IPython

In [ ]:
checkpoint = './checkpoints/exp0/states_val_loss=19.79108.ckpt'
hparams_file = './configs/hparams.yaml'
device = 'cuda'

model = EncDecCTCModel.load_from_checkpoint(
    checkpoint, hparams_file=hparams_file)
model = model.to(device)
model.eval();

In [ ]:
audio_path = './data/test_waves/static1.wav'

audio, sr = torchaudio.load(audio_path)
audio = audio.to(device)
if sr != 16000:
    audio = torchaudio.functional.resample(audio, sr, 16000)
audio /= audio.abs().max()
audio.squeeze_(0)

text_pred = stt(model, audio)

IPython.display.display(
    IPython.display.Audio(data=0.5*audio.cpu(), rate=16000, normalize=False),
)
print(text_pred)


In [ ]:
import torch
import matplotlib.pyplot as plt
from utils.letters import letters_ds_voc

signal_len = torch.LongTensor([len(audio)]).to(audio.device)
mel, _ = model.preprocessor.get_features(audio[None], signal_len)
text_pred, logits = stt(model, audio, return_logits=True)

fig, (axl, axr) = plt.subplots(1, 2, figsize=(9, 3))
axl.imshow(mel[0].cpu(), aspect='auto', origin='lower', interpolation='none')
axr.imshow(logits[0].cpu().t(), aspect='auto', origin='lower', interpolation='none')

print(list(enumerate(letters_ds_voc)))